
# Исследование базы данных StackOverflow 

##  Исходные данные 

 Самостоятельная работа основана на базе данных  StackOverflow — сервиса вопросов и ответов о программировании. 
 Мы будем работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах есть информация о более поздних оценках, которые эти посты получили. 

## Цель исследования 

Проект состоит из двух частей:

В первой части несколько задач в SQL-тренажёре, чтобы закрепить пройденный материал.

Вторая часть проекта — аналитическая.

## Задания Часть 1 

### Задание 1 

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
'''
SELECT COUNT(id)
FROM stackoverflow.posts  
WHERE post_type_id = 1  AND (score > 300 OR  favorites_count >= 100 )
GROUP BY post_type_id; 
'''

### Задание 2 

Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
'''
SELECT ROUND(AVG (p.count) , 0)
FROM  (SELECT COUNT(id), 
       creation_date:: date
FROM stackoverflow.posts
WHERE post_type_id = 1 
GROUP BY  creation_date :: date  
HAVING creation_date :: date BETWEEN '2008-11-01' AND '2008-11-18') AS p;
'''

### Задание 3

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
'''
SELECT COUNT(DISTINCT b.user_id)
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS us ON us.id = b.user_id 
WHERE us.creation_date :: date = b.creation_date :: date;
'''

### Задание 4

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
'''
SELECT COUNT (DISTINCT n.id)
FROM (SELECT p.id
FROM stackoverflow.posts AS p 
JOIN stackoverflow.votes AS v  ON  p.id = v.post_id 
JOIN stackoverflow.users AS u ON p.user_id = u.id 
WHERE u.display_name = 'Joel Coehoorn' AND v.id > 0  ) AS n;
'''

### Задание 5

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
'''
SELECT *,
ROW_NUMBER () OVER (ORDER BY id DESC) as rnk
FROM stackoverflow.vote_types
ORDER BY id; 
'''

### Задание 6 

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
'''
SELECT *
FROM ( SELECT v.user_id, 
          COUNT (vt.id) AS vt_count
    FROM stackoverflow.votes AS v 
    JOIN stackoverflow.vote_types AS vt ON vt.id = v.vote_type_id
    WHERE vt.name LIKE 'Close'
    GROUP BY v.user_id 
    ORDER BY vt.count DESC
    LIMIT 10 ) AS p
ORDER BY p.vt_count DESC, p.user_id DESC; 
'''

### Задание 7

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:

  *  идентификатор пользователя;
  *  число значков;
  *  место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
'''
SELECT *, 
     DENSE_RANK () OVER (ORDER BY l.b_count DESC) AS rating
FROM (SELECT user_id, 
       COUNT(id) AS b_count
FROM stackoverflow.badges AS b 
WHERE creation_date :: date BETWEEN '2008-11-15' AND '2008-12-15' 
GROUP BY user_id 
ORDER BY b_count DESC, user_id ASC 
LIMIT 10 ) AS l; 
'''

### Задание 8 

Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:

   * заголовок поста;
   * идентификатор пользователя;
   * число очков поста;
   * среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков. 

In [ ]:
'''
WITH temp_table AS  ( SELECT  ROUND(AVG(score)) AS avg_score, 
                 user_id
                 FROM stackoverflow.posts 
                WHERE title IS NOT NULL AND score != 0
                 GROUP BY user_id) 
SELECT p.title, 
       temp_table.user_id, 
       p.score, 
       temp_table.avg_score
FROM temp_table 
JOIN stackoverflow.posts AS p ON temp_table.user_id = p.user_id 
WHERE p.title IS NOT NULL AND p.score != 0 ;
'''  

### Задание 9 


Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
'''
SELECT title
FROM stackoverflow.posts
WHERE user_id IN (SELECT user_id 
FROM stackoverflow.badges 
GROUP BY user_id 
HAVING COUNT(id) > 1000) AND title IS NOT NULL ;
''' 

### Задание 10

Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

  *  пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
  *  пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
  *  пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
'''
SELECT id, 
       views, 
       CASE 
           WHEN views >= 350 THEN 1 
           WHEN views < 100 THEN 3 
           ELSE 2 
       END AS group 
FROM stackoverflow.users 
WHERE location LIKE '%Canada%' AND views > 0 ;
'''

### Задание 11 

Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
'''
WITH temp_table AS (SELECT h.id, 
                           h.views, 
                           h.row,
                     MAX(h.views) OVER (PARTITION BY h.row) AS max
            FROM ( SELECT id, 
                          views, 
                          CASE
                              WHEN views >= 350 THEN 1 
                              WHEN views < 100 THEN 3 
                              ELSE 2 
                          END AS row
                   FROM stackoverflow.users 
                   WHERE location LIKE '%Canada%' AND views > 0 ) AS h )
SELECT temp_table.id ,
       temp_table.views, 
       temp_table.row
FROM temp_table 
WHERE temp_table.views = temp_table.max 
ORDER BY temp_table.views DESC, temp_table.id ASC; 
'''

### Задание 12 

Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:

   * номер дня;
   * число пользователей, зарегистрированных в этот день;
   * сумму пользователей с накоплением.

In [ ]:
'''
SELECT *, 
       SUM(tr.count_of_reg) OVER (ORDER BY tr.number_of_day) AS count_users 
FROM (SELECT EXTRACT(DAY FROM CAST(creation_date AS date)) as number_of_day,
       COUNT (id) AS count_of_reg
FROM stackoverflow.users 
WHERE creation_date :: date BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY EXTRACT (DAY FROM CAST(creation_date AS date))) AS tr; 
'''

### Задание 13 

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:

  *  идентификатор пользователя;
  *  разницу во времени между регистрацией и первым постом.

In [ ]:
'''
WITH temp_table AS (SELECT  DISTINCT user_id,
                   MIN(creation_date)  OVER (PARTITION BY user_id) AS min_time 
                    FROM stackoverflow.posts)
SELECT temp_table.user_id, 
       (temp_table.min_time - u.creation_date) AS diff
FROM stackoverflow.users AS u 
JOIN temp_table ON u.id = temp_table.user_id;
'''

### Задания Часть 2 

### Задание 1 

Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
'''
SELECT SUM(views_count), 
        CAST(DATE_TRUNC('month', creation_date) AS date)  as month
FROM stackoverflow.posts 
GROUP BY CAST(DATE_TRUNC('month', creation_date) AS date) 
ORDER BY SUM(views_count) DESC;
'''

### Задание 2

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
'''
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id 
JOIN stackoverflow.post_types AS pt ON pt.id = p.post_type_id
WHERE p.creation_date :: date BETWEEN u.creation_date  :: date AND (u.creation_date:: date + INTERVAL '1 month')
           AND pt.type LIKE '%Answer%'
GROUP BY u.display_name 
HAVING COUNT(p.id) > 100 
ORDER BY u.display_name ASC;
'''

### Задание 3 

Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
'''
WITH temp_table AS (SELECT u.id
                  FROM stackoverflow.posts AS p 
                  JOIN stackoverflow.users AS u ON u.id = p.user_id 
                   WHERE CAST (DATE_TRUNC('month', u.creation_date) AS date) = '2008-09-01'
                    AND CAST(DATE_TRUNC('month', p.creation_date) AS date) = '2008-12-01' 
                   GROUP BY u.id 
                   HAVING COUNT (p.id) > 0) 
 SELECT COUNT(p.id), 
        CAST(DATE_TRUNC('month', p.creation_date) AS date) 
        FROM stackoverflow.posts AS p 
        WHERE p.user_id IN (SELECT * 
                            FROM temp_table ) 
        AND CAST(DATE_TRUNC('year', p.creation_date) AS date) = '2008-01-01' 
  GROUP BY CAST(DATE_TRUNC('month', p.creation_date) AS date)
  ORDER BY CAST(DATE_TRUNC('month', p.creation_date) AS date) DESC; 
'''    
                            

### Задание 4 

Используя данные о постах, выведите несколько полей:

  *  идентификатор пользователя, который написал пост;
  *  дата создания поста;
  *  количество просмотров у текущего поста;
  *  сумма просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
'''
SELECT user_id,
       creation_date, 
       views_count,
       SUM(views_count) OVER( PARTITION BY user_id ORDER BY creation_date) AS sum_count
FROM stackoverflow.posts ; 
'''

### Задание 5 

In [ ]:
'''
WITH temp_table AS ( SELECT user_id, 
                     COUNT(DISTINCT creation_date :: date) AS count
                   FROM stackoverflow.posts 
                    WHERE creation_date :: date  between '2008-12-01' AND '2008-12-07' 
                    GROUP BY user_id) 
                    
SELECT ROUND(AVG(temp_table.count))
FROM temp_table; 
'''

### Задание 6 

На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:

   * Номер месяца.
   * Количество постов за месяц.
   * Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
'''
WITH temp_table AS (SELECT EXTRACT(MONTH FROM creation_date :: date) AS month, 
               COUNT(distinct id)
               FROM stackoverflow.posts 
               WHERE creation_date :: date BETWEEN '2008-09-01' AND '2008-12-31' 
               GROUP BY month) 
SELECT *,
       ROUND(((count :: numeric / LAG(count) OVER (ORDER BY month)) - 1 ) * 100,2) AS percentage_change
      FROM temp_table;
'''

### Задание 7 


Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:

  * номер недели;
  * дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
'''
WITH table1 AS ( SELECT user_id ,
                 COUNT(DISTINCT id) AS count
                FROM stackoverflow.posts
                 GROUP BY user_id 
                ORDER BY count DESC 
                LIMIT 1) , 
      table2 AS ( SELECT p.user_id, 
                        p.creation_date, 
                  EXTRACT('week' FROM p.creation_date ) AS week_number 
                 FROM stackoverflow.posts AS p
                 JOIN table1 ON table1.user_id = p.user_id
                 WHERE DATE_TRUNC('month', p.creation_date) :: date ='2008-10-01')
 SELECT DISTINCT week_number:: numeric, 
       MAX(creation_date) OVER (PARTITION BY week_number) AS data_post
         FROM table2
         ORDER BY week_number;
'''                

                 